# **Part 1: Query Matching**

In [1]:
!pip install thefuzz -q
!pip install thefuzz scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1.1: Importing Libraries

In [2]:
import pandas as pd
from thefuzz import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 1.2: Loading Input Data

In [3]:
unresolved_tickets = pd.read_csv("new_queries.csv")
knowledge_base = pd.read_csv("resolved_queries.csv")

In [4]:
unresolved_tickets.head(), knowledge_base.head()

(                             Variation_Query  Matches_With_Query_ID
 0           Unabel to conect to the internet                      1
 1                  Can’t connect to internet                      1
 2                        Intenet not working                      1
 3               Payment failed while chekout                      2
 4  Payment did not go through during chckout                      2,
    Query_ID                    Pre_Resolved_Query
 0         1     Unable to connect to the internet
 1         2        Payment failed during checkout
 2         3     App crashes when opening settings
 3         4   Forgot password and unable to reset
 4         5  Unable to upload files to the server)

### 1.3: Text Normalization

In [5]:
# Clean the text data by converting to lowercase and removing leading/trailing spaces
unresolved_tickets["Variation_Query"] = unresolved_tickets["Variation_Query"].str.lower().str.strip()
knowledge_base["Pre_Resolved_Query"] = knowledge_base["Pre_Resolved_Query"].str.lower().str.strip()

### 1.4: Fuzzy String Matching Analysis

In [6]:
def find_best_fuzzy_match(query_text, kb_df, min_score=70):
    """Iterates through different fuzzy scorers to find the best match."""
    known_queries = kb_df["Pre_Resolved_Query"].tolist()
    scorers = {
        "ratio": fuzz.ratio,
        "partial_ratio": fuzz.partial_ratio,
        "token_sort_ratio": fuzz.token_sort_ratio,
        "token_set_ratio": fuzz.token_set_ratio
    }
    
    best_score = -1
    best_match_text = None
    best_method_name = None

    for method_name, scorer_func in scorers.items():
        match_text, score = process.extractOne(query_text, known_queries, scorer=scorer_func)
        if score > best_score:
            best_score = score
            best_match_text = match_text
            best_method_name = method_name

    if best_score >= min_score:
        matched_id = kb_df.loc[kb_df["Pre_Resolved_Query"] == best_match_text, "Query_ID"].iloc[0]
        return best_method_name, best_match_text, matched_id, best_score
    
    return None, None, None, None

# Apply the fuzzy matching logic
fuzzy_match_results = []
for query in unresolved_tickets["Variation_Query"]:
    result = find_best_fuzzy_match(query, knowledge_base)
    fuzzy_match_results.append((query,) + result)

fuzzy_analysis_df = pd.DataFrame(fuzzy_match_results, columns=[
    "Unresolved_Query", "Best_Method", "Fuzzy_Match", "Fuzzy_Query_ID", "Fuzzy_Score"
])

### 1.5: TF-IDF and Cosine Similarity Analysis

In [7]:
vectorizer_model = TfidfVectorizer(stop_words="english")

# Create a corpus of all queries
corpus = unresolved_tickets["Variation_Query"].tolist() + knowledge_base["Pre_Resolved_Query"].tolist()
tfidf_vectors = vectorizer_model.fit_transform(corpus)

# Split the vectors back into unresolved and resolved sets
num_unresolved = len(unresolved_tickets)
unresolved_vectors = tfidf_vectors[:num_unresolved]
resolved_vectors = tfidf_vectors[num_unresolved:]

# Calculate cosine similarity
similarity_matrix = cosine_similarity(unresolved_vectors, resolved_vectors)

tfidf_match_results = []
for i, query in enumerate(unresolved_tickets["Variation_Query"]):
    top_match_index = similarity_matrix[i].argmax()
    top_score = similarity_matrix[i][top_match_index]
    matched_info = knowledge_base.iloc[top_match_index]
    tfidf_match_results.append((query, matched_info["Pre_Resolved_Query"], matched_info["Query_ID"], top_score))

tfidf_analysis_df = pd.DataFrame(tfidf_match_results, columns=[
    "Unresolved_Query", "TFIDF_Match", "TFIDF_Query_ID", "TFIDF_Score"
])

### 1.6: Merging Results from Both Methods

In [8]:
merged_analyses = pd.merge(fuzzy_analysis_df, tfidf_analysis_df, on="Unresolved_Query")

### 1.7: Final Selection Logic

In [9]:
def select_best_match(data_row, f_thresh=75, t_thresh=0.65):
    """Applies thresholds to pick the final match, prioritizing fuzzy search."""
    fuzzy_score = data_row["Fuzzy_Score"]
    tfidf_score = data_row["TFIDF_Score"]

    if fuzzy_score and fuzzy_score >= f_thresh:
        return pd.Series([data_row["Fuzzy_Match"], data_row["Fuzzy_Query_ID"], f"Fuzzy-{data_row['Best_Method']}"])
    elif tfidf_score >= t_thresh:
        return pd.Series([data_row["TFIDF_Match"], data_row["TFIDF_Query_ID"], "TFIDF"])
    else:
        return pd.Series([None, None, "No Match"])

final_assignments = merged_analyses.apply(select_best_match, axis=1)
final_assignments.columns = ["Final_Match", "Final_Query_ID", "Method_Used"]

final_results = pd.concat([merged_analyses, final_assignments], axis=1)

### 1.8: Display Final Output

In [10]:
# The final output is structured to match the original assignment's requested columns.
# Note: Internally, we used 'final_results', but the display requires the original 'combined' name for consistency.
combined = final_results.rename(columns={'Final_Match': 'Fuzzy_Match'}) # Rename for display purposes
combined[['Unresolved_Query', 'Fuzzy_Match', 'Method_Used']]

,Unresolved_Query,Fuzzy_Match,Fuzzy_Match,Method_Used
0,unabel to conect to the internet,unable to connect to the internet,unable to connect to the internet,Fuzzy-ratio
1,can’t connect to internet,unable to connect to the internet,unable to connect to the internet,Fuzzy-token_set_ratio
2,intenet not working,None,None,No Match
3,payment failed while chekout,payment failed during checkout,payment failed during checkout,Fuzzy-ratio
4,payment did not go through during chckout,payment failed during checkout,None,No Match
5,payment issue at check out,payment failed during checkout,None,No Match
6,application crashes when opening setings,app crashes when opening settings,app crashes when opening settings,Fuzzy-partial_ratio
7,app crash when going to settings,app crashes when opening settings,app crashes when opening settings,Fuzzy-ratio
8,settings cause the app to chrash,None,None,No Match
9,forgot passwrd and cant reset,forgot password and unable to reset,forgot password and unable to reset,Fuzzy-ratio


# **Part 2: Name Variation Matching**

### 2.1: Importing Libraries

In [11]:
import re

### 2.2: Data Ingestion

In [12]:
variant_names_df = pd.read_csv("name_variations.csv")
canonical_names_df = pd.read_csv("base_names.csv")

In [13]:
canonical_names_df.head(), variant_names_df.head()

(   Base_Name_ID         Base_Name
 0             1        John Smith
 1             2    Jennifer Brown
 2             3  Michael O'Connor
 3             4      Maria Garcia
 4             5        Robert Lee,
       Variation Matches_With_Base_Name
 0  Thomas  King            Thomas King
 1    ThomasKing            Thomas King
 2  Maria Garcia           Maria Garcia
 3     MaryLewis             Mary Lewis
 4      Nancy W.           Nancy Wright)

### 2.3: Name Cleaning and Standardization Function

In [14]:
def clean_and_standardize_name(input_str: str) -> str:
    """Standardizes a name string for effective matching."""
    if not isinstance(input_str, str):
        return ""
    # Add space for camelCase names, e.g., 'ThomasKing' -> 'Thomas King'
    processed_str = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', input_str)
    # Convert to lowercase and remove non-alphabetic chars
    processed_str = re.sub(r'[^a-z\s]', '', processed_str.lower())
    # Normalize whitespace
    processed_str = re.sub(r'\s+', ' ', processed_str).strip()
    return processed_str

### 2.4: Applying the Standardization

In [15]:
variant_names_df["Standardized_Name"] = variant_names_df["Variation"].apply(clean_and_standardize_name)
canonical_names_df["Standardized_Name"] = canonical_names_df["Base_Name"].apply(clean_and_standardize_name)

### 2.5: Matching Logic using token_sort_ratio

In [16]:
def find_canonical_match(standardized_name, canonical_df, min_confidence=80):
    """Finds the best matching canonical name for a given standardized variant."""
    choices = canonical_df["Standardized_Name"]
    
    # Use token_sort_ratio as it handles reordered names like 'Smith John'
    best_match = process.extractOne(
        standardized_name,
        choices,
        scorer=fuzz.token_sort_ratio
    )

    if best_match and best_match[1] >= min_confidence:
        # Retrieve the original, un-standardized base name
        original_name = canonical_df.loc[canonical_df["Standardized_Name"] == best_match[0], "Base_Name"].iloc[0]
        return original_name, best_match[1]
    
    return None, None

### 2.6: Executing the Matching Process

In [17]:
match_data = [
    (original_variant, *find_canonical_match(standardized_variant, canonical_names_df))
    for original_variant, standardized_variant in zip(variant_names_df["Variation"], variant_names_df["Standardized_Name"])
]

### 2.7: Compiling Results into a DataFrame

In [18]:
matches_df = pd.DataFrame(match_data, columns=["Variation_Name", "Matched_Base_Name", "Score"])
matches_df.head(15)

,Variation_Name,Matched_Base_Name,Score
0,Thomas King,Thomas King,100.0
1,ThomasKing,Thomas King,100.0
2,Maria Garcia,Maria Garcia,100.0
3,MaryLewis,Mary Lewis,100.0
4,Nancy W.,None,NaN
5,Dani3l Scott,Daniel Scott,96.0
6,JOHN smith,John Smith,100.0
7,linda johnson,Linda Johnson,100.0
8,N@ncy Wright,Nancy Wright,96.0
9,William Davis,William Davis,100.0


### 2.8: Performance Evaluation

In [19]:
from sklearn.metrics import accuracy_score

# Prepare ground truth and predictions for comparison
ground_truth = variant_names_df['Matches_With_Base_Name'].fillna("No Match")
model_predictions = matches_df['Matched_Base_Name'].fillna("No Match")

# Calculate accuracy
model_accuracy = accuracy_score(ground_truth, model_predictions)
model_accuracy

0.95